# Fitzy (Product Recommendation System)

### Importing Libraries

In [1]:
import tensorflow as tf
import numpy as np
from keras.utils import load_img, img_to_array
from keras.applications.resnet import ResNet50,preprocess_input
from keras.layers import GlobalMaxPooling2D
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle

### Creating Model

In [2]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3)) # rgb color(3), top layer setting false.
model.trainable = False
model = tf.keras.Sequential([
  model,
  GlobalMaxPooling2D()
])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-01-01 15:37:45.031204: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-01 15:37:45.031479: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Function for extracting the Image feature

In [3]:
def extract_features(img_path, model):
    img = load_img(img_path, target_size=(224, 224, 3))
    img_array = img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)
    return normalized_result

In [4]:
filenames = []
# adding file path and file to this array
for file in os.listdir('images'):
  filenames.append(os.path.join('images',file))
  
features_list = []
# extracting each image feature with model
for file in filenames:
  features_list.append(extract_features(img_path=file,model=model))
  

2023-01-01 15:37:46.427188: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-01 15:37:46.703670: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 36ms/step


### Exporting the model

In [5]:
pickle.dump(features_list,open('model/trained_image_set.pkl','wb'))
pickle.dump(filenames,open('model/filenames.pkl','wb'))